# Dataset Bitcoin

# Présentation et objectifs

Le Bitcoin est la plus ancienne crypto-monnaie, publiée pour la première fois en open source en 2009 par Satoshi Nakamoto. C'est un support décentralisé d'échange numérique, avec des transactions vérifiées et enregistrées publiquement (blockchain).L'intéret réside dans le fait qu'il n'y a pas besoin d'une autorité de tenue de dossiers  ou d'un intermédiaire central. Les blocs de transaction contiennent un hachage cryptographique des blocs de transaction précédents et sont donc "enchaînés" ensemble, cela sert d'enregistrement de toutes les transactions. Comme pour toute devise, le trading de bitcoins et les instruments financiers ont rapidement suivi l'adoption publique du bitcoin. 





Le but de ce notebook est de prédire les prix du Bitcoin en s'aidant de différents modèles prédictifs, afin de determiner lequel est le plus performant.<br>
La difficulté réside dans le fait que le bitcoin est tres volatile.<br>
On procédera tout d'abord à une analyse exploratoire des données puis une analyse des séries temporelles avant d'introduire différents modeles prédictifs

# Analyse Exploratoire

On dispose de 2 fichiers csv qui determinent les transactions de bitcoin dans une période définie entre le 31/12/2011 et le 22/04/2020 avec des mises a jour minute par minute.<br>
L'horodatage se fait en temps Unix

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import de librairies

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats
from itertools import product
import warnings

warnings.filterwarnings('ignore')

## Chargement des fichiers csv et visualisation des premieres valeurs

In [ ]:
df = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv')
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv')

In [ ]:
print('Dimension du 1er fichier',df.shape)
print('Dimension du 2eme fichier',df2.shape)

In [ ]:
frames=[df, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)
print(data.shape)
data.head(3)

## Conversion du Timestamp

In [ ]:
import datetime, pytz
#define a conversion function for the native timestamps in the csv file
def dateparse (time_in_secs):    
    return pytz.utc.localize(datetime.datetime.fromtimestamp(float(time_in_secs)))


data1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
data2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)

## Infos sur les dates des deux dataframes

In [ ]:
# intervalle de temps concerné data1
print(data1.Timestamp.min(), data1.Timestamp.max())

#intervalle data2
print(data2.Timestamp.min(), data2.Timestamp.max())

## Fusion des deux dataframes et suppression des valeurs dupliquées

In [ ]:
frames=[data1, data2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)
print(data.shape)
data.head(3)

In [ ]:
data.info()

Les données du dataframe comporte 6,43 millions des lignes et de 8 variables informant sur les prix du bitcoin a un instant précis.

In [ ]:
#visualisation
data=data.sort_values(by='Timestamp', ascending=True)
data_corr=data.copy()
data.head(3)


In [ ]:
# Génére des grosse erreurs (modifier le temps pris en compte par 3 courbes journalieres (min/moyenne/max))
'''import plotly.express as px

fig = px.line(data, x='Timestamp', y='Weighted_Price')
fig.show()
'''

Prétraitement: remplacement des NaN par des zéros et des données suivantes.

In [ ]:
# On remplace les nan par la valeur 0 quand il n'y a pas de transaction de bitcoin
data['Volume_(BTC)'].fillna(value=0, inplace=True)
data['Volume_(Currency)'].fillna(value=0, inplace=True)
data['Weighted_Price'].fillna(value=0, inplace=True)

# on attribue des valeurs aux données OHLC qui sont des données continues
# on remplace les valeurs manquantes avec la suivante
data['Open'].fillna(method='backfill', inplace=True)
data['High'].fillna(method='backfill', inplace=True)
data['Low'].fillna(method='backfill', inplace=True)
data['Close'].fillna(method='backfill', inplace=True)

In [ ]:
# conversion timestamp
data.Timestamp = pd.to_datetime(data.Timestamp, unit='s')

# conversion jours
data.index = data.Timestamp
data = data.resample('D').mean()

#conversion mois
data_month = data.resample('M').mean()

#conversion année
data_year = data.resample('A-DEC').mean()

In [ ]:
data.head()

In [ ]:
# intervalle de temps concerné df
data.index.min(), data.index.max()

### Visualisation des variation des prix du bitcoin en fonction de la date 

In [ ]:
#variation du prix du bitcoin (1er df)
fig = plt.figure(figsize=(20,5))
#variation quotidienne
plt.subplot(131)
plt.plot(data.Weighted_Price, '-', label='Quotidien')
plt.legend()
#variation mensuelle
plt.subplot(132)
plt.plot(data_month.Weighted_Price, '-', label='Mensuel')
plt.legend()
#variation annuelle
plt.subplot(133)
plt.plot(data_year.Weighted_Price, '-', label='Annuel')
plt.legend()
plt.suptitle('Variation des prix du bitcoin')
# plt.tight_layout()
plt.show()

In [ ]:
# variation quotidienne du prix du bitcoin pour le 1er df
fig = plt.figure(figsize=(20,5))
plt.plot(data.Weighted_Price, '-', label='Quotidien')
plt.legend()
plt.suptitle('Variation quotidienne des prix du bitcoin')
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
data.High.plot(kind='line',color='g',label='high',linewidth=1,alpha=0.5,grid=True,linestyle=':')
data.Low.plot(color='r',label='Low',linewidth=1,alpha=0.5,linestyle='-.',grid=True)
plt.legend('upper right')
plt.suptitle('Bitcoin')
plt.show()

## Interprétation graphique de la valeur du bitcoin: 
- Stable entre 2011 et 2017;
- Forte augmentation durant l'année 2017 jusqu'a atteindre sa valeur maximale fin 2017 (pres de 20000 dollars/unité);
- Forte chute début 2018, puis stagnation 7500 dollars/unité dans le courant d'année et nouvelle chute fin d'année;
- Hausse début 2019 et depuis mai "stabilisation" 

## Un bref historique du bitcoin
- 2008, le  domaine bitcoin.org a été enregistré.
- novembre 2010, le capital social de Bitcoin atteint 1 million dollars. Son taux de change: 0,50 USD par BTC.
- juin 2011, taux 10 USD/BTC. 
- mars 2013, taux 32 USD/BTC.
- avril 2013, taux 100 USD/BTC.
- février 2015, taux 260 USD/BTC.
- janvier 2017, franchit taux 1000 USD/BTC
- juin 2017, taux dépasse 3000 USD/BTC.
- decembre 2017, taux 10000 USD/BTC.
- fin décembre 2017, Bitcoin atteint un niveau record, mais n'atteint pas 20 000 USD.
- 28 décembre 2017, le taux du bitcoin chute apres une aznnonce de la Corée du Sud du controle du prix 
- novembre 2018, taux 6300 USD/BT

## Corrélation entre les variables

In [ ]:
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(data_corr.corr(),annot=True,linewidths=.5,fmt='.1f',ax=ax)
plt.show()

## Décomposition STL (tendence/saisonnalité/résidu) et visualisation 

In [ ]:
# meilleure visualisation de decomposition STL
plt.style.use('seaborn-poster')

#plt.figure(figsize=(20,15))
sm.tsa.seasonal_decompose(data_month.Weighted_Price).plot()
plt.title('decomposition saison du df')
plt.show()

## Interprétation de la décomposition:
on constate effectivement beaucoup de bruit dans l'annalyse saisonnière lors des periodes de forte variatoin de prix

In [ ]:
# test de dickey fuller
print("Dickey–Fuller test: p=%f" % sm.tsa.stattools.adfuller(data_month.Weighted_Price)[1])
# Transformtion Box-Cox 
data_month['Weighted_Price_box'], lmbda = stats.boxcox(data_month.Weighted_Price)
print("Dickey–Fuller test: p=%f" % sm.tsa.stattools.adfuller(data_month.Weighted_Price)[1])

In [ ]:
data.head()

Les séries temporelles ont plusieurs caractéristiques clés telles que la tendance, la saisonnalité et le bruit.La prévision est le processus de prédiction de l'avenir, basé sur les données passées et présentes.

Dans ce notebook, nous tentons d'effectuer une analyse de séries temporelles sur les données historiques du prix du Bitcoin. On voit, que les prix du Bitcoin étaient assez volatils et incohérents avec le temps. Il est très difficile d'effectuer une analyse de séries chronologiques sur de telles données volatiles. Mais ici, on essaye d'explorer différents modèles de prévision de séries temporelles.

- Prévision de séries chronologiques avec XGBoost
- Prévision de séries chronologiques avec ARIMA
- Prévision de séries chronologiques avec Facebook Prophet


# Modèles prédictifs

# XGBoost
XGBoost est une implémentation d'arbres de décision à gradient amélioré conçus pour sa vitesse et ses performances. On regarde dans quelle mesure XGBoost fonctionne pour prédire les valeurs de cette série temporelle

In [ ]:
# fonction definition recherche croisée
def my_xgb(data, target, params):
    
    # crée X & y
    X = data.drop(target, axis=1)
    y = data[target]
    
    # mise a l'échelle X
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    
    # XGBoost classifier 
    xgb_clf = xgb.XGBClassifier()
    
    # recherche grille 
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    gridsearch = GridSearchCV(xgb_clf, param_grid=params, scoring="accuracy", cv=kf, return_train_score=True)
    gridsearch.fit(X_scaled, y)
    
    # Return the gridsearch results plus the scaler
    return gridsearch, scaler

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
df1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)

In [ ]:
frames=[df1, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)

In [ ]:
#regle Timestamp en fonction des heures
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)
data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
data = data.set_index('Timestamp')
data = data[['Weighted_Price']]
data['Weighted_Price'].fillna(method='backfill', inplace=True)

In [ ]:
# graphe de variation des prix train/test sets
plt.style.use('fivethirtyeight')
_ = data.plot(style='', figsize=(15,5), title='Prix du bitcoin(donnés des heure)')

## Selection de date limite train/test set

In [ ]:
#detemination de la date limite train/test set suivant le ratio 70/30
print(data.shape)
data.head(51000)


On fixe le 25 octobre 2017 comme date de séparation cette date représentant le sueil du ratio 70/30. Et on introduit les données de test et d'entrainement

In [ ]:
# selection de la date de séparation des train/test sets
split_date = '25-Oct-2017'
data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [ ]:
# graphe de variation des prix train/test sets
_ = data_test.rename(columns={'Weighted_Price': 'Test Set'}).join(data_train.rename(columns={'Weighted_Price': 'Training Set'}), how='outer').plot(figsize=(15,5), title='Variation du prix du bitcoin(donnés des heure)', style='')

On peut douter de la qualité de cette selection de date, tant elle n'est pas représentative de la tendance entre 2018 et 2020. La prédiction qui en aurait découlé n'aurait fait que croitre. <br>
Il s'agit de choisir un date plus cohérente, deux intervalles pourraeint convenir, celui entre aout et novembre 2018 et un autre vers mi 2019.
On choisit une date dans l'un de ces deux intervalles (je choisis le 31 octobre 2018 -date des 10 ans du bitcoin-)

## Autre date de test 31/10/2018 - 10 ans du Bitcoin

In [ ]:
# selection de la date de séparation des train/test sets
split_date = '31-Oct-2018'
data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [ ]:
# graphe de variation des prix train/test sets
_ = data_test.rename(columns={'Weighted_Price': 'Test Set'}).join(data_train.rename(columns={'Weighted_Price': 'Training Set'}), how='outer').plot(figsize=(15,5), title='Variation du prix du bitcoin(donnés des heure)', style='')

In [ ]:
#Creation fonction timeseries a partir dun index de date

def create_features(df, label=None):
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
# introduction de la configuration du XGBoost
X_train, y_train = create_features(data_train, label='Weighted_Price')
X_test, y_test = create_features(data_test, label='Weighted_Price')

In [ ]:
# prediction XGBoost

import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier


params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3 )

In [ ]:
model =  xgb.XGBRegressor(objective ='reg:squarederror',min_child_weight=10, booster='gbtree', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 100)

model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=50, verbose=True)

In [ ]:
# jointure des données test/train
data_test['Weighted_Price_Prediction'] = model.predict(X_test)
data_all = pd.concat([data_test, data_train], sort=False)
data_all

In [ ]:
#introduction données finales pour comparer les modeles
final_data = data_all
final_data = final_data.reset_index()

In [ ]:
#Comparaison entre modeles
final_data = pd.merge(final_data, data_all, sort=False)
final_data = final_data.rename(columns={'Weighted_Price_Prediction': 'xgboost'})
final_data = final_data[['Timestamp','Weighted_Price','xgboost']]
final_data

In [ ]:
#visualisation prédiction/réelles
_ = data_all[['Weighted_Price','Weighted_Price_Prediction']].plot(figsize=(15, 5))

## Autre date: 04/07/2019 

In [ ]:
# selection de la date de séparation des train/test sets
split_date = '04-Jul-2019'
data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [ ]:
# graphe de variation des prix train/test sets
_ = data_test.rename(columns={'Weighted_Price': 'Test Set'}).join(data_train.rename(columns={'Weighted_Price': 'Training Set'}), how='outer').plot(figsize=(15,5), title='Variation du prix du bitcoin(donnés des heure)', style='')

In [ ]:
# introduction de la configuration du XGBoost
X_train, y_train = create_features(data_train, label='Weighted_Price')
X_test, y_test = create_features(data_test, label='Weighted_Price')

In [ ]:
# prediction XGBoost

import xgboost as xgb
from xgboost import plot_importance, plot_tree

model =  xgb.XGBRegressor(objective ='reg:squarederror',min_child_weight=10, booster='gbtree', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 100)

model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=50, verbose=True)

In [ ]:
# jointure des données test/train
data_test['Weighted_Price_Prediction'] = model.predict(X_test)
data_all = pd.concat([data_test, data_train], sort=False)
data_all

In [ ]:
#visualisation prédiction/réelles
_ = data_all[['Weighted_Price','Weighted_Price_Prediction']].plot(figsize=(15, 5))

# Arima

On a clairement vu que la série temporelle n'estt pas stationnaire.
Il faut essayer de la rendre stationnaire c'est a dire que la moyenne de la variance reste constante en fonction du temps. On utilise un test de Dickey-Fuller pour savoir la stationarité ou non.

Guide pour les séries temporelles: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/

Nous utilisons le modèle ARIMA pour analyser d'abord nos séries chronologiques. Nous savons que pour cela, nous avons besoin que notre série soit stationnaire. Nous utilisons donc les techniques décrites ci-dessous pour réaliser si notre série est stationnaire:
-  décomposition saisonnière pour visualiser les composantes saisonnières et tendancielles des séries chronologiques. Nous visons à obtenir un résidu indépendant de tendances et de saisonnalité.
- test de Dicky Fuller considère l'hypothèse nulle que la série chronologique considérée n'est pas stationnaire. Si la valeur de p est suffisamment faible (inférieure à 0,05) lors du test d'hypothèse, alors seulement nous rejetons l'hypothèse nulle et on peut considérons la série comme stationnaire

In [ ]:
from statsmodels.tsa.stattools import adfuller
from scipy.stats import boxcox

### test Dickey-Fuller
def DFTest(series):
    testdf = adfuller(series)
    print("DF test p-value : %.16f" %testdf[1] )

In [ ]:
from scipy import stats
import statsmodels.api as sm
import warnings
from itertools import product

In [ ]:
df1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)
frames=[df1, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)

In [ ]:
data['Open'].fillna(method='backfill', inplace=True)
data['High'].fillna(method='backfill', inplace=True)
data['Low'].fillna(method='backfill', inplace=True)
data['Close'].fillna(method='backfill', inplace=True)
data['Weighted_Price'].fillna(method='backfill', inplace=True)
data['Volume_(BTC)'].fillna(method='backfill', inplace=True)
data['Volume_(Currency)'].fillna(method='backfill', inplace=True)

On s'interesse a la saisonalité donc on utilisera les données mensuelles

In [ ]:
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)
data = data.groupby([pd.Grouper(key='Timestamp', freq='M')]).first().reset_index()
data = data.set_index('Timestamp')
data['Weighted_Price'].fillna(method='backfill', inplace=True)

In [ ]:
plt.figure(figsize=[20,8])
plt.title('Variation prix bicoin par mois')
plt.plot(data.Weighted_Price, '-', label='By Months')
plt.show()

In [ ]:
#plt.figure(figsize=(20,15))
sm.tsa.seasonal_decompose(data_month.Weighted_Price).plot()
plt.title('decomposition saison du df')
plt.show()

In [ ]:
print("Test Dickey–Fuller : p=%f" % sm.tsa.stattools.adfuller(data.Weighted_Price)[1])
DFTest(data.Weighted_Price)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
pyplot.figure(figsize=(20,8))
pyplot.subplot(211)
plot_acf(data.Weighted_Price, ax=pyplot.gca(),lags=40)
pyplot.subplot(212)
plot_pacf(data.Weighted_Price, ax=pyplot.gca(), lags=50)
pyplot.show()

### transformation logarithmique

In [ ]:
prices = data.Weighted_Price
prices_log = np.log(prices)
DFTest(prices_log)

On obtient une valeur de p=0.3 avec le Test de DF donc on ne peut pas considérer la série comme stationnaire

In [ ]:
prices_log_r = prices_log - prices_log.shift(12)
prices_log_r.dropna(inplace = True)

DFTest(prices_log_r)

#### Transformation de Box-Cox

In [ ]:
prices_box_cox_, lambda_ = boxcox(prices)
prices_box_cox = pd.Series(data = prices_box_cox_, index = data.index) 
DFTest(prices_box_cox)
print('lambda value:', lambda_)


In [ ]:
prices_box_cox_r = prices_box_cox - prices_box_cox.shift(12)
prices_box_cox_r.dropna(inplace = True)

DFTest(prices_box_cox_r)


Maintenant qu'on a obtenu des résultats satisfaisants au test de Dickey-Fuller, tracons les graphes ACF/PACF des dfonctions transformées afin d'avoir une idée de comment âramétrer Arima

Maintenant qu'on a obtenue la stationnarirté, il faut faire une selection de modele

Fonction d'autocorrélation - ACF: Le graphique résume la corrélation d'une observation avec des valeurs de décalage. L'axe des x montre le décalage et l'axe des y montre le coefficient de corrélation entre -1 et 1 pour la corrélation négative et positive
Fonction d'autocorrélation partielle - PACF: Le graphique résume les corrélations d'une observation avec des valeurs de décalage qui ne sont pas prises en compte par les observations antérieures. On peut obtenir une image de base de l'intervalle de paramètres, puis décider quels sont les meilleurs p, q, d pour ARIMA

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

plt.figure(figsize = (14,7)) 
a = acf(prices_log_r)
p = pacf(prices_log_r)

plt.subplot(221)
sns.lineplot(data = a)
plt.axhline(y=0, linestyle='--', color='gray')
plt.grid(linestyle='dotted')
plt.title('Courbe ACF')

plt.subplot(222)
sns.lineplot(data = p)
plt.axhline(y=0, linestyle='--', color='gray')
plt.grid(linestyle='dotted')
plt.title('Courbe PACF')
plt.show()

### interprétation (doc: https://people.duke.edu/~rnau/411arim.htm)
Nous déduisons du graphique que L'ACF et le PACF se rapprochent de zéro tandis que le décalage approche 1.<br>
Essai différentes valeurs de p et q <br>
D = 1

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

a = [[1,2,3], [1],[1,2,3]]
params = list(product(*a))

results = []   
min_aic = float('inf')
best_param = []

# checking different set of params for best fit
for param in params:
    try:
        model = ARIMA(prices_log, order = param).fit(disp = -1)
    except LinAlgError:
        print('Rejected Parameters:', param)
        continue
    except ValueError:
        print('Rejected Parameters:', param)
        continue
    if(min_aic > model.aic):
        min_aic = model.aic
        best_param = param
        best_model = model
        
    results.append([param, model.aic])

print(best_param,min_aic)
print(results)

print(best_model.fittedvalues)

plt.figure(figsize=(16,8))
sns.lineplot(data = prices_log_r, color = 'blue')
sns.lineplot(data = best_model.fittedvalues, color = 'red')
plt.grid(linestyle='dotted')
plt.title('Valeurs du meilleur modele (rouge) VS. valeur prix apres trandformation log')
plt.show()

In [ ]:
fitted_values = best_model.fittedvalues
fitted_values = fitted_values.cumsum()

fitted_values = fitted_values + prices_log[0]

final_values = np.exp(fitted_values)

d = {'prices' : prices, 'prices_log' : prices_log, 'price_log_r' : prices_log_r, 'fitted_values' : fitted_values, 'final_values' : final_values}
summaryDF = pd.DataFrame(data = d)

plt.figure(figsize=(16,8))
sns.lineplot(data = summaryDF['prices'], color = 'blue', label='réelle')
sns.lineplot(data = summaryDF['final_values'], color = 'red', label='predite')
plt.grid(linestyle='dotted')
plt.title('valeurs prédites(rouge) / valeurs réelles')
plt.show()

In [ ]:
# Initialisation parametres
Qs = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
ps = range(0, 3)
D=1
d=1
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

# Selection Modele
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
    try:
        model=sm.tsa.statespace.SARIMAX(data.Weighted_Price, order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], D, param[3], 12),enforce_stationarity=False,
                                            enforce_invertibility=False).fit(disp=-1)
    except ValueError:
        #print('wrong parameters:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
# Meilleurs modeles
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())
print(best_model.summary())

In [ ]:
print("Test Dickey–Fuller : p=%f" % sm.tsa.stattools.adfuller(best_model.resid)[1])

In [ ]:
#figure résidus meilleur modele
fig = plt.figure(figsize=(20,8))
best_model.resid.plot()
fig.suptitle('Residus meilleur modele')
plt.show()

In [ ]:
#graphe comparatif précdiction valeurs réelles
df_month2 = data[['Weighted_Price']]
future = pd.DataFrame()
df_month2 = pd.concat([df_month2, future])
df_month2['forecast'] = best_model.predict(start=0, end=200)
plt.figure(figsize=(15,7))
df_month2.Weighted_Price.plot()
df_month2.forecast.plot(color='r', ls='--', label='Predicted Weighted_Price')
plt.legend()
plt.title('Comparatif Prediction/Données réelles des variation du bitcoin ')
plt.ylabel('$')
plt.show()

### Prophet

Prophet est un modele prédictif développé par Facebook de time seies. Il est basé sur un modèle additif où les tendances non linéaires sont adaptées aux saisonnalité quotidiennes, annuelles et hebdomadaires, ainsi que les effets des vacances.<br>
Le fonctionnement de Prophet est s'améliore des séries temporelles qui ont de forts effets saisonniers et plusieurs saisons de données historiques. Prophet résiste aux données manquantes et aux changements de tendance, et gère généralement bien les valeurs aberrantes. <br>
Doc Prophet: [Fonctionnement et documentation Prophet](https://facebook.github.io/prophet/)


In [ ]:
df1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)
frames=[df1, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)

In [ ]:
#regle Timestamp en fonction des heures
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)
data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
data = data.set_index('Timestamp')
data = data[['Weighted_Price']]
data['Weighted_Price'].fillna(method='backfill', inplace=True)

In [ ]:
_ = data.plot(style='', figsize=(15,5), title='Prix du bitcoin(donnés des heure)')

#### Split date

In [ ]:
split_date = '04-Jul-2018'
data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [ ]:
# graphe de variation des prix train/test sets
_ = data_test.rename(columns={'Weighted_Price': 'Test Set'}).join(data_train.rename(columns={'Weighted_Price': 'Training Set'}), how='outer').plot(figsize=(15,5), title='Variation du prix du bitcoin(donnés des heure)', style='')

### Configuration Prophet:
On doit nécéssairement avoir un dataframe composé de deux colonnes 'ds' et 'y'.<br>
ds correspondant au datestamp et y devant correspondre a une valeur numérique et représente la variable qu'on veut prédire

In [ ]:
#import de la librairie Prophet
from fbprophet import Prophet

In [ ]:
# configuration du modele d'entrainement
# On renomme les colonnes selon les normes de Prophet
data_train = data_train.reset_index().rename(columns={'Timestamp':'ds', 'Weighted_Price':'y'})

m = Prophet()
m.fit(data_train)

In [ ]:
# prediction sur l'ensemble training set
data_test_fcst = m.predict(df=data_test.reset_index().rename(columns={'Timestamp':'ds'}))

In [ ]:
# graphe de la prediction 
f, ax = plt.subplots()
f.set_figheight(12)
f.set_figwidth(15)
fig = m.plot(data_test_fcst, ax=ax)

In [ ]:
# composantes temporelles
fig = m.plot_components(data_test_fcst)

In [ ]:
# graphe prédiction/réelles
f, ax = plt.subplots(1)
f.set_figheight(12)
f.set_figwidth(15)
ax.scatter(data_test.index, data_test['Weighted_Price'], color='r')
fig = m.plot(data_test_fcst, ax=ax)

## Remarques:

Manifestement sur l'ensemble des modeles qu'on a introduit, on n'aobtient pas de résultat tres significatifs.<br>
Pour obtenir de meilleurs résultats on peut imaginer deux hypotheses: 
- intervalle de données plus réduit
- prediction sur une courte période (10 jours/1 mois)

In [ ]:
start_date='01-Apr-2018'
split_date = '01-Feb-2020'

## XGBoost

In [ ]:
data_boost=data.copy()

In [ ]:
#regle Timestamp en fonction des heures
data_boost['Timestamp'] = data_boost['Timestamp'].dt.tz_localize(None)
data_boost = data_boost.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
data_boost = data_boost.set_index('Timestamp')
data_boost = data_boost[['Weighted_Price']]
data_boost['Weighted_Price'].fillna(method='backfill', inplace=True)

In [ ]:
data_train = data_boost.loc[(data_boost.index <= split_date) & (data_boost.index > start_date)].copy()
data_test = data_boost.loc[(data_boost.index > split_date)].copy()

In [ ]:
# introduction de la configuration du XGBoost
X_train, y_train = create_features(data_train, label='Weighted_Price')
X_test, y_test = create_features(data_test, label='Weighted_Price')

In [ ]:
model =  xgb.XGBRegressor(objective ='reg:squarederror',min_child_weight=10, booster='gbtree', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 100)

model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=50, verbose=True)

In [ ]:
# jointure des données test/train
data_test['Weighted_Price_Prediction'] = model.predict(X_test)
data_all = pd.concat([data_test, data_train], sort=False)
data_all

In [ ]:
#Comparaison entre modeles
final_data = pd.merge(final_data, data_all, sort=False)
final_data = final_data.rename(columns={'Weighted_Price_Prediction': 'xgboost'})
final_data = final_data[['Timestamp','Weighted_Price','xgboost']]
final_data

In [ ]:
#visualisation prédiction/réelles
_ = data_all[['Weighted_Price','Weighted_Price_Prediction']].plot(figsize=(15, 5))

## ARIMA

In [ ]:
df1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)
frames=[df1, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)
raw_data=data.copy()

In [ ]:
data['Open'].fillna(method='backfill', inplace=True)
data['High'].fillna(method='backfill', inplace=True)
data['Low'].fillna(method='backfill', inplace=True)
data['Close'].fillna(method='backfill', inplace=True)
data['Weighted_Price'].fillna(method='backfill', inplace=True)
data['Volume_(BTC)'].fillna(method='backfill', inplace=True)
data['Volume_(Currency)'].fillna(method='backfill', inplace=True)

In [ ]:
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)
data = data.groupby([pd.Grouper(key='Timestamp', freq='M')]).first().reset_index()
data = data.set_index('Timestamp')
data['Weighted_Price'].fillna(method='backfill', inplace=True)

In [ ]:
data=data.loc[data.index > start_date]

In [ ]:
plt.figure(figsize=[20,8])
plt.title('Variation prix bicoin par mois')
plt.plot(data.Weighted_Price, '-', label='By Months')
plt.show()

In [ ]:
#plt.figure(figsize=(20,15))
sm.tsa.seasonal_decompose(data_month.Weighted_Price).plot()
plt.title('decomposition saison du df')
plt.show()

In [ ]:
print("Test Dickey–Fuller : p=%f" % sm.tsa.stattools.adfuller(data.Weighted_Price)[1])
DFTest(data.Weighted_Price)

In [ ]:
prices = data.Weighted_Price
prices_log = np.log(prices)
DFTest(prices_log)

In [ ]:
prices_box_cox_, lambda_ = boxcox(prices)
prices_box_cox = pd.Series(data = prices_box_cox_, index = data.index) 
DFTest(prices_box_cox)
print('lambda value:', lambda_)

In [ ]:
prices_box_cox_r = prices_box_cox - prices_box_cox.shift(12)
prices_box_cox_r.dropna(inplace = True)

DFTest(prices_box_cox_r)

In [ ]:
a = [[1,2,3], [1],[1,2,3]]
params = list(product(*a))

results = []   
min_aic = float('inf')
best_param = []

# checking different set of params for best fit
for param in params:
    try:
        model = ARIMA(prices_log, order = param).fit(disp = -1)
    except LinAlgError:
        print('Rejected Parameters:', param)
        continue
    except ValueError:
        print('Rejected Parameters:', param)
        continue
    if(min_aic > model.aic):
        min_aic = model.aic
        best_param = param
        best_model = model
        
    results.append([param, model.aic])

print(best_param,min_aic)
print(results)

print(best_model.fittedvalues)

plt.figure(figsize=(16,8))
sns.lineplot(data = prices_log_r, color = 'blue')
sns.lineplot(data = best_model.fittedvalues, color = 'red')
plt.grid(linestyle='dotted')
plt.title('Valeurs du meilleur modele (rouge) VS. valeur prix apres trandformation log')
plt.show()

In [ ]:
fitted_values = best_model.fittedvalues
fitted_values = fitted_values.cumsum()

fitted_values = fitted_values + prices_log[0]

final_values = np.exp(fitted_values)

d = {'prices' : prices, 'prices_log' : prices_log, 'price_log_r' : prices_log_r, 'fitted_values' : fitted_values, 'final_values' : final_values}
summaryDF = pd.DataFrame(data = d)

plt.figure(figsize=(16,8))
sns.lineplot(data = summaryDF['prices'], color = 'blue', label='réelle')
sns.lineplot(data = summaryDF['final_values'], color = 'red', label='predite')
plt.grid(linestyle='dotted')
plt.title('valeurs prédites(rouge) / valeurs réelles')
plt.show()

In [ ]:
# Initialisation parametres
Qs = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
ps = range(0, 3)
D=1
d=1
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

# Selection Modele
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
    try:
        model=sm.tsa.statespace.SARIMAX(data.Weighted_Price, order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], D, param[3], 12),enforce_stationarity=False,
                                            enforce_invertibility=False).fit(disp=-1)
    except ValueError:
        #print('wrong parameters:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
# Meilleurs modeles
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())
print(best_model.summary())

In [ ]:
print("Test Dickey–Fuller : p=%f" % sm.tsa.stattools.adfuller(best_model.resid)[1])

In [ ]:
#figure résidus meilleur modele
fig = plt.figure(figsize=(20,8))
best_model.resid.plot()
fig.suptitle('Residus meilleur modele')
plt.show()

In [ ]:
#graphe comparatif précdiction valeurs réelles
df_month2 = data[['Weighted_Price']]
future = pd.DataFrame()
df_month2 = pd.concat([df_month2, future])
df_month2['forecast'] = best_model.predict(start=0, end=200)
plt.figure(figsize=(15,7))
df_month2.Weighted_Price.plot()
df_month2.forecast.plot(color='r', ls='--', label='Predicted Weighted_Price')
plt.legend()
plt.title('Comparatif Prediction/Données réelles des variation du bitcoin ')
plt.ylabel('$')
plt.show()

## Prophet

Au vu des résultats des prédictions précédentes qui ne sont pas d'une précision suffisante, on choisit de ne considérer que les valeurs a partir de 2018, et on observera les résultats des valeurs prédites du prix du Bitcoin

In [ ]:
df1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)
frames=[df1, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)
raw_data=data.copy()

In [ ]:
data.info()

In [ ]:
#regle Timestamp en fonction des heures
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)
data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
data = data.set_index('Timestamp')
data = data[['Weighted_Price']]
data['Weighted_Price'].fillna(method='backfill', inplace=True)

In [ ]:
data=data.loc[data.index > start_date]

In [ ]:
_ = data.plot(style='', figsize=(15,5), title='Prix du bitcoin(donnés des heure)')

In [ ]:

data_train = data.loc[(data.index <= split_date)].copy()
data_test = data.loc[data.index > split_date].copy()

In [ ]:
# graphe de variation des prix train/test sets
_ = data_test.rename(columns={'Weighted_Price': 'Test Set'}).join(data_train.rename(columns={'Weighted_Price': 'Training Set'}), how='outer').plot(figsize=(15,5), title='Variation du prix du bitcoin(donnés des heure)', style='')

In [ ]:
# configuration du modele d'entrainement
# On renomme les colonnes selon les normes de Prophet
data_train = data_train.reset_index().rename(columns={'Timestamp':'ds', 'Weighted_Price':'y'})

m = Prophet()
m.fit(data_train)

In [ ]:
# prediction sur l'ensemble training set
data_test_fcst = m.predict(df=data_test.reset_index().rename(columns={'Timestamp':'ds'}))

In [ ]:
# graphe de la prediction 
f, ax = plt.subplots()
f.set_figheight(8)
f.set_figwidth(15)
fig = m.plot(data_test_fcst, ax=ax)

In [ ]:
# composantes temporelles
fig = m.plot_components(data_test_fcst)

In [ ]:
# graphe prédiction/réelles
f, ax = plt.subplots(1)
f.set_figheight(8)
f.set_figwidth(15)
ax.scatter(data_test.index, data_test['Weighted_Price'], color='r')
fig = m.plot(data_test_fcst, ax=ax)

 # Deuxieme modele 

# Tests

In [ ]:
start_date='01-Apr-2019'
split_date = '01-Apr-2020'

In [ ]:
df1 = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv', parse_dates=[0], date_parser=dateparse)
df2 = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv', parse_dates=[0], date_parser=dateparse)
frames=[df1, df2]
data=pd.concat(frames).drop_duplicates().reset_index(drop=True)
raw_data=data.copy()

# Tests
pas assez de RAM !


In [ ]:
# erreur intentionnelle 
x=

In [ ]:

data_test=raw_data.copy()
data_test['Timestamp'] = data_test['Timestamp'].dt.tz_localize(None)

In [ ]:
data_test

In [ ]:
data_test.rename(columns={'Timestamp': 'ds', 'Weighted_Price': 'y'},inplace=True)
data_test['ds'] = pd.to_datetime(data_test['ds'])
data_test['y']=data_test['y'].astype(float)

In [ ]:
data_test.head(1)

In [ ]:
import datetime
import pytz
import datetime

x = datetime.datetime.now()
x

In [ ]:
#Fit the model 
df_prophet = Prophet(changepoint_prior_scale=0.15, daily_seasonality=True)
df_prophet.fit(data_test)

In [ ]:
fcast_time=365   # 1 year
df_forecast = df_prophet.make_future_dataframe(periods= fcast_time, freq='D')
df_forecast